In [4]:
# import standard libraries
import pandas as pd
import numpy as np

# import spacy for NLP and re for regular expressions
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import re

# import sklearn transformers, models and pipelines
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import train_test_split

# Load the small language model from spacy
nlp = spacy.load('en_core_web_sm')

# set pandas text output to 400
pd.options.display.max_colwidth = 400

In [2]:
df_fin_phrase = pd.read_csv("../../Data/Prepared/NormalAnalysis/fin_phrase_bank_clean.csv")

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_fin_phrase['clean_text'], df_fin_phrase['label'], test_size=0.3, random_state=42)

In [22]:
# Load the en_core_web_lg model
nlp = spacy.load('en_core_web_lg', disable=["tagger", "parser", "ner"])
docs_train = [nlp(str(doc)).vector for doc in X_train]
X_train = np.vstack(docs_train)
print('Shape of train set: {}'.format(X_train.shape))


Shape of train set: (3392, 300)


In [23]:
docs_test = [nlp(doc).vector for doc in X_test]
X_test = np.vstack(docs_test)
print('Shape of test set: {}'.format(X_test.shape))

/home/juann/anaconda3/envs/DDB/lib/python3.11/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


Shape of test set: (1454, 300)


In [29]:
word2vec_pipe = Pipeline([('estimator', LogisticRegression(C=1e3, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4, max_iter=1000))])



In [31]:
# cross validate
print('F1 score: {:.3f}'.format(np.mean(cross_val_score(word2vec_pipe, X_train, y_train, scoring = 'f1_micro'))))

/home/juann/anaconda3/envs/DDB/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/juann/anaconda3/envs/DDB/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

F1 score: 0.689


In [32]:
# fit pipeline
word2vec_pipe.fit(X_train, y_train)

# predict on test set
pred = word2vec_pipe.predict(X_test)



/home/juann/anaconda3/envs/DDB/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
X_train, X_test, y_train, y_test = train_test_split(df_fin_phrase['clean_text'], df_fin_phrase['label'], test_size=0.3, random_state=42)

In [35]:
df = pd.DataFrame({'text': X_test, 'true_label': y_test, 'pred_label': pred})

In [38]:
df.to_csv('../../Data/Prepared/Word2vecAnalysis/FinancialWord2vecLogicReg.csv', index=False)